In [ ]:
import re
import pandas as pd
import numpy as np
import Wizard as oz

In [2]:
# network_csv = pd.read_csv(input('Enter Filename:'))  #Read in file,error checking will come later. For now, I trust me.
network_csv = pd.read_csv('HPG_HPA.csv')
network_csv

,RelationSymbolicName,ObjectType,Effect,Mechanism,RelationNumberOfReferences,RelationConfidenceLevel,msrc
0,MolSynthesis: FST ---> progesterone,MolSynthesis,NaN,NaN,10,3,"As for ID{4000000,4108202=steroidogenesis}, ID..."
1,positive MolSynthesis: epinephrine --+> proges...,MolSynthesis,positive,NaN,7,3,It was reported that ID{1271077=epinephrine} a...
2,positive MolSynthesis: Luteinizing hormone --+...,MolSynthesis,positive,NaN,53,3,Social contact stimulates the secretion of ID{...
3,negative MolSynthesis: cortisol ---| ACTH,MolSynthesis,negative,NaN,46,3,"Finally, it has been previously shown that ID{..."
4,positive MolSynthesis: CRH --+> cortisol,MolSynthesis,positive,NaN,73,3,"ID{10000000,8802722=Placental} ID{1392=CRH} ma..."
5,positive MolSynthesis: cortisol --+> epinephrine,MolSynthesis,positive,NaN,9,3,ID{1100010=Cortisol} promotes the synthesis of...
6,negative MolSynthesis: CRH ---| progesterone,MolSynthesis,negative,NaN,15,3,Because ID{1392=corticotropin-releasing factor...
7,positive MolSynthesis: norepinephrine --+> pro...,MolSynthesis,positive,NaN,10,3,ID{1267744=norepinephrine} acts on ID{154=β2-a...
8,positive MolSynthesis: cortisol --+> norepinep...,MolSynthesis,positive,NaN,6,3,"The rise in ID{10000000,6800052=plasma} ID{110..."
9,positive MolSynthesis: FSH --+> estrogen,MolSynthesis,positive,NaN,133,3,ID{3000051=follicle-stimulating hormone} stimu...


In [3]:
#Conform Data
network = pd.DataFrame(columns=['Source', 'Target', 'Data'])

for i in network_csv.index:
    strLine = network_csv.iloc[i][0]
    m = re.search('(?<=: ).*', strLine)
    strLine = m.group()
    m = re.search('.*(?= --)', strLine)
    df_source = m.group()
    m = re.search('(?<=[>|] ).*', strLine)
    df_target = m.group()
    df_data = network_csv.iloc[i][6]

    network.loc[i] = [df_source, df_target, df_data]

In [4]:
network[4] = network_csv.RelationNumberOfReferences
network2 = network

In [5]:
#Consolodate Duplicates and  Data
network = network.groupby(['Source', 'Target'])['Data'].apply(' '.join).reset_index()
for idx, row in network.iterrows():
    strLine = row.Data
    strLine = re.sub('\|',' ',strLine)
    strLine = re.sub('\"', '', strLine)
    strLine = re.sub('  ', ' ', strLine)
    row.Data = strLine

In [6]:
network

,Source,Target,Data
0,ACTH,CRH,Blocking ID{1392=CRH}-stimulated increases in ...
1,ACTH,GNRH1,The increased levels of ID{1392=CRH} and ID{14...
2,ACTH,cortisol,"Besides, different types of ID{9000000,9016100..."
3,ACTH,epinephrine,"CRF then causes the ID{10000000,8803038=pituit..."
4,ACTH,progesterone,ID{1409770=Adrenocorticotropic hormone} stimul...
5,AVP,ACTH,ID{551=AVP} binding to ID{553=V1bR} in the ID{...
6,AVP,CRH,ID{551=AVP} may also directly increase secreti...
7,AVP,KISS1,Several lines of evidence have suggested that ...
8,AVP,OXT,We mention here that ID{1267744=norepinephrine...
9,AVP,cortisol,Several studies have demonstrated that ID{551=...


In [7]:
splitter = oz.Splitter()
tagger = oz.POSTagger()
dict_tag = oz.DictionaryTagger(['postive.yml','negative.yml','inv.yml'])

In [8]:
#Sentiment Scoring, possibly add weight to individual words [promote, inhibit]
score = []
for chunk in network['Data']:
    split_sentences = splitter.split(chunk)
    pos_sentences = tagger.pos_tag(split_sentences)
    tagged_sentences = dict_tag.tag(pos_sentences)
    relationship_score = oz.sentiment_score(tagged_sentences)
    
    score.append(relationship_score)

In [9]:
network['Sentiment Score'] = score
polarity = []
for sc in score:
    if sc > 0:
        polarity.append('Positive')
    elif sc < 0:
        polarity.append('Negative')
    else:
        polarity.append('Unknown')

network['Polarity'] = polarity

In [10]:
network

,Source,Target,Data,Sentiment Score,Polarity
0,ACTH,CRH,Blocking ID{1392=CRH}-stimulated increases in ...,1.0,Positive
1,ACTH,GNRH1,The increased levels of ID{1392=CRH} and ID{14...,-6.0,Negative
2,ACTH,cortisol,"Besides, different types of ID{9000000,9016100...",97.0,Positive
3,ACTH,epinephrine,"CRF then causes the ID{10000000,8803038=pituit...",2.0,Positive
4,ACTH,progesterone,ID{1409770=Adrenocorticotropic hormone} stimul...,19.0,Positive
5,AVP,ACTH,ID{551=AVP} binding to ID{553=V1bR} in the ID{...,33.0,Positive
6,AVP,CRH,ID{551=AVP} may also directly increase secreti...,1.0,Positive
7,AVP,KISS1,Several lines of evidence have suggested that ...,3.0,Positive
8,AVP,OXT,We mention here that ID{1267744=norepinephrine...,0.0,Unknown
9,AVP,cortisol,Several studies have demonstrated that ID{551=...,31.0,Positive


In [11]:
# Created weighted sentiment
weighted_score = []
for chunk in network['Data']:
    split_sentences = splitter.split(chunk)
    pos_sentences = tagger.pos_tag(split_sentences)
    tagged_sentences = dict_tag.tag(pos_sentences)
    relationship_score = oz.weighted_sentiment_score(tagged_sentences, False)
    weighted_score.append(relationship_score)

In [12]:
network['Weighted Sentiment Score'] = weighted_score
weighted_polarity = []
for sc in weighted_score:
    if sc > 0.5:
        weighted_polarity.append('Positive')
    elif sc < -0.5:
        weighted_polarity.append('Negative')
    else:
        weighted_polarity.append('Unknown')

network['Weighted Polarity'] = weighted_polarity

In [13]:
network.head(5)

,Source,Target,Data,Sentiment Score,Polarity,Weighted Sentiment Score,Weighted Polarity
0,ACTH,CRH,Blocking ID{1392=CRH}-stimulated increases in ...,1.0,Positive,0.061453,Unknown
1,ACTH,GNRH1,The increased levels of ID{1392=CRH} and ID{14...,-6.0,Negative,-0.119358,Unknown
2,ACTH,cortisol,"Besides, different types of ID{9000000,9016100...",97.0,Positive,2.079108,Positive
3,ACTH,epinephrine,"CRF then causes the ID{10000000,8803038=pituit...",2.0,Positive,0.040884,Unknown
4,ACTH,progesterone,ID{1409770=Adrenocorticotropic hormone} stimul...,19.0,Positive,0.514539,Positive


In [14]:
network[network['Polarity'] != 'Unknown'].reset_index().drop('Data',1).to_csv(path_or_buf='HPG_HPA_Sentiment.csv',
                                                                              columns= ['Source','Target','Polarity'],index=False)